# Loading the libraries

In [33]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /usr/share/java/postgresql-42.2.23.jar --jars /usr/share/java/postgresql-42.2.23.jar pyspark-shell'
import databricks.koalas as ks
import numpy as np
import pandas as pd
import networkx as nx
from heapq import heapify, heapreplace#, heappop, heappush

# Splitting the lengthy edges

In [34]:
params = {'user': 'cristiano', 'password': 'cristiano'}

def loadMultiGraph():
	kdf = ks.read_sql_query('''	select	EDGE.IDVERTEXORIG_FK,
										EDGE.IDVERTEXDEST_FK,
										EDGE.IDEDGE,
										EDGE.LENGTH,
										EDGE.ONEWAY
								from	STREETSEGMENT as EDGE
								where	EDGE.UTILITYVALUE <> 0 ''',
					'jdbc:postgresql:afterqualifying', options=params)

	G = nx.MultiGraph()
	for row in kdf.itertuples():
		dictRow = row._asdict()

		G.add_edge(dictRow['idvertexdest_fk'], dictRow['idvertexorig_fk'], key=dictRow['idedge'], idedge=dictRow['idedge'], length=dictRow['length'])

	#G = nx.from_pandas_edgelist(kdf.to_pandas(), 'idvertexorig_fk', 'idvertexdest_fk', ['idedge', 'length'], create_using=nx.MultiGraph(), edge_key='idedge')
	
	return G

#G = loadMultiGraph()

In [35]:
def reBuildGraph(G, edgesHeap, firstSplit):
    for item in edgesHeap:
        (heapValue, u, v, idedge, lengthOriginal, numSplit) = item
        #The number of segments the edge must be split into is 1 less the value stored in the heap
        numSplit = numSplit - 1
        if numSplit >= firstSplit:
            lengthSplitted = lengthOriginal/numSplit
            vertexStart = u

            #print(G[u][v][idedge]['length'], nx.dijkstra_path_length(G, u, v, weight='length'), numSplit, lengthSplitted, lengthOriginal)
            oldDistance = nx.dijkstra_path_length(G, u, v, weight='length')

            G.remove_edge(u, v, key=int(idedge))
            for i in range(numSplit - 1):
                vertexEnd = str(idedge) + '_' + str(i + 1)
                G.add_edge(vertexStart, vertexEnd, key=vertexEnd, idedge=vertexEnd, length=lengthSplitted)
                vertexStart = vertexEnd
            keyLast = str(idedge) + '_' + str(numSplit)
            G.add_edge(vertexStart, v, key=keyLast, idedge=keyLast, length=lengthSplitted)

            #print(nx.dijkstra_path_length(G, u, v, weight='length'))
            newDistance = nx.dijkstra_path_length(G, u, v, weight='length')
            if round(oldDistance, 7) != round(newDistance, 7):
                print("ERROR IN DISTANCES:", oldDistance, newDistance)

    return G

In [36]:
def splitEdges(precision=1, maxDistance=None):
    G = loadMultiGraph()

    firstSplit = 2
    #The value must be negative because the data structure is a min heap
    edgesHeap = [(-1*data['length'], u, v, data['idedge'], data['length'], firstSplit) for u, v, data in G.edges(data=True)]
    heapify(edgesHeap)
    
    newLength = G.number_of_edges()
    if precision != 0:
        for i in range(len(edgesHeap) * precision):
            #(heapValue, u, v, idedge, lengthOriginal, numSplit) = heappop(edgesHeap)
            (heapValue, u, v, idedge, lengthOriginal, numSplit) = edgesHeap[0]

            #The value must be negative because the data structure is a min heap
            heapValue = -1 * lengthOriginal/numSplit

            #The numSplit is prepared for the next time the edge may be splitted (numsplit + 1)
            heapreplace(edgesHeap, (heapValue, u, v, idedge, lengthOriginal, numSplit + 1))

            newLength += 1
            #The value must be multiplied by -1 because the data structure is a min heap
            if maxDistance != None and -1 * edgesHeap[0][0] <= maxDistance:
                break

        #reBuildGraph(G, edgesHeap, firstSplit)

    lengths = sorted([-1 * item[0] for item in edgesHeap])

    return lengths, newLength

data, newLength = splitEdges(0)
data = np.asarray(data)
print(0, newLength)
data = data.reshape((data.shape[0], 1))
maxPrecision = 11
for i in range(1, maxPrecision):
    newData, newLength = splitEdges(i)
    newData = np.asarray(newData)
    print(i, newLength)
    newData = newData.reshape((newData.shape[0], 1))
    data = np.concatenate((data, newData), axis=1)

kdf = ks.DataFrame(data=data, columns=range(maxPrecision))
kdf.describe()

0 285186
1 570372
2 855558
3 1140744
4 1425930
5 1711116
6 1996302
7 2281488
8 2566674
9 2851860
10 3137046


,0,1,2,3,4,5,6,7,8,9,10
count,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000,285186.000000
mean,71.224519,31.913708,21.508227,16.348612,13.221870,11.103385,9.573581,8.416820,7.512525,6.782742,6.185991
std,66.791126,11.858228,6.014225,3.731162,2.590161,1.931411,1.504580,1.216844,1.010400,0.854861,0.731310
min,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059,0.015059
25%,28.043877,25.670794,19.141893,15.083587,12.432172,10.585200,9.226715,8.171300,7.335570,6.644001,6.078606
50%,58.426013,34.319274,23.197015,17.523705,14.070580,11.755174,10.083288,8.829732,7.856299,7.076001,6.436121
75%,96.566379,41.049938,25.986626,18.997345,14.986014,12.378948,10.536989,9.183338,8.137403,7.300828,6.623124
max,6703.573236,49.084935,28.700760,20.356114,15.820669,12.933786,10.953637,9.497866,8.377583,7.498051,6.784633


In [37]:
data, newLength = splitEdges(maxDistance=500)
print(-1, newLength)
kdfDistance = ks.DataFrame(data)
kdfDistance.describe()

-1 285607


,0
count,285186.000000
mean,70.661664
std,58.069957
min,0.015059
25%,28.043877
50%,58.426013
75%,96.566379
max,499.613211


In [38]:
def formatFloat(value):
    numDecimaPlaces = 4
    return " & " + "{:.{nDigits}f}".format(value, nDigits=numDecimaPlaces)

stringKdf = kdf.describe().to_string(float_format=formatFloat).replace("\n", " \\\\\n")
print(stringKdf)

stringKdf = kdfDistance.describe().to_string(float_format=formatFloat).replace("\n", " \\\\\n")
print(stringKdf)

                  0              1              2              3              4              5              6              7              8              9              10 \\
count  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000  & 285186.0000 \\
mean       & 71.2245      & 31.9137      & 21.5082      & 16.3486      & 13.2219      & 11.1034       & 9.5736       & 8.4168       & 7.5125       & 6.7827       & 6.1860 \\
std        & 66.7911      & 11.8582       & 6.0142       & 3.7312       & 2.5902       & 1.9314       & 1.5046       & 1.2168       & 1.0104       & 0.8549       & 0.7313 \\
min         & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151       & 0.0151 \\
25%        & 28.0439      & 25.6708      & 19.1419      & 15.0836      & 12.4322      & 10.5852       & 9.2267       & 8.1713     